In [1]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocess as mp
import glob
import time
from tqdm import tqdm
import os
import sys
import pandas as pd
import dbfread 
import geopandas as gpd
import warnings
import matplotlib.pyplot as plt
import csv
import pyproj
from simpledbf import Dbf5
from datetime import datetime

In [2]:
# Read the desired sheet by sheet name or index (replace 'Sheet1' with the actual sheet name or index)
# gemas = pd.read_csv('/mnt/diskstation/data/Soil_points/EU/GEMAS/GEMAS.csv')
# liguriaa = Dbf5('/home/opengeohub/xuemeng/work_xuemeng/SOC density-spatial-temporal-model/Italy_soil/Liguria/Original_data/Profili_liguria.dbf')
# liguria = liguriaa.to_dataframe()
# sloveniaa = Dbf5('/mnt/diskstation/data/Soil_points/Slovenia/Horizonti_v_PedoloskihProfilih.DBF', encoding='latin-1')
# slovenia = sloveniaa.to_dataframe()
# print(len(slovenia))

In [17]:
lucas = pd.read_csv('/home/opengeohub/xuemeng/work_xuemeng/spatiotemporal-soc-eu/lucas_original.csv',low_memory=False)
lucas = lucas.loc[:, ~lucas.columns.str.contains('^Unnamed')]
print(f'{len(lucas)} data in lucas original')

# get lucas with depth
dep_lucas = lucas.loc[~lucas['OC (20-30 cm)'].isna()]
col = ['POINT_ID', 'OC (20-30 cm)', 'GPS_LAT', 'GPS_LONG', 'time']
dep_lucas = dep_lucas[col]
dep_lucas = dep_lucas.rename(columns={'OC (20-30 cm)':'OC'})
dep_lucas['N'] = np.nan
dep_lucas['Depth'] = '20-30cm'
dep_lucas['pH_in_CaCl2'] = np.nan
dep_lucas['pH_in_H2O'] = np.nan
dep_lucas
print(f'{len(dep_lucas)} data in 20-30cm')

col = ['POINT_ID', 'OC', 'N', 'GPS_LAT', 'GPS_LONG', 
       'time', 'Depth', 'pH_in_CaCl2', 'pH_in_H2O']
lucas = lucas[col]
df = pd.concat([lucas,dep_lucas])

df['POINT_ID'] = pd.to_numeric(df['POINT_ID'], errors='coerce')
df = df.rename(columns={'POINT_ID':'point_id',
                        'OC':'oc','GPS_LAT':'soil_lat','GPS_LONG':'soil_long','Depth':'depth'})
df = df.sort_values('point_id')
print(f'{len(df)} data in total, of all depth')

62846 data in lucas original
141 data in 20-30cm
62987 data in total, of all depth


In [22]:
# get the real coordinates
df = df.reset_index()
tt = [2009,2012,2015,2018]
df['lat'] = 0
df['lc_survey'] = 0
df['lon'] = 0
df['nuts0'] = 0
for i in tt:
    l = pd.read_csv(f'/home/opengeohub/xuemeng/work_xuemeng/delta_soc/lucas_microdata/EU_{i}_20200213.CSV',low_memory=False)
    l = l[['GPS_LAT','GPS_LONG','POINT_ID','LC1','NUTS0']]
    l['POINT_ID'] = pd.to_numeric(l['POINT_ID'], errors='coerce')
    l = l.rename(columns={'POINT_ID':'point_id'})
    df = df.merge(l,how='left',on='point_id')
    df.loc[df['time']==i,'lat'] = df.loc[df['time']==i,'GPS_LAT']
    df.loc[df['time']==i,'lon'] = df.loc[df['time']==i,'GPS_LONG']
    df.loc[df['time']==i,'nuts0'] = df.loc[df['time']==i,'NUTS0']
    df.loc[df['time']==i,'lc_survey'] = df.loc[df['time']==i,'LC1']
    df = df.drop(columns=['GPS_LAT','GPS_LONG','LC1','NUTS0'])
    
df.loc[(df['soil_lat']<0) & (df['lat']>0),'lat'] = df.loc[(df['soil_lat']<0) & (df['lat']>0)]['lat']* -1
df.loc[(df['soil_long']<0) & (df['lon']>0),'lon'] = df.loc[(df['soil_long']<0) & (df['lon']>0)]['lon']* -1
print(len(df))

62987


In [37]:
lucas = df

depth_lucas = lucas.loc[~lucas['depth'].isna()]
nod_lucas = lucas.loc[lucas['depth'].isna()]
# lucas point without specifying depth info
column_names = ['ph_h2o','ph_ca','oc','gps_lat','gps_long','time','hzn_top','hzn_bottom','ref','ph_kcl']
temp = pd.DataFrame(columns=column_names)
temp['oc'] = nod_lucas['oc']
temp['ph_kcl'] = np.nan
temp['ph_h2o'] = nod_lucas['pH_in_H2O']
temp['ph_ca'] = nod_lucas['pH_in_CaCl2']
temp['time'] = nod_lucas['time']
temp['hzn_top'] = 0
temp['hzn_bottom'] = 20
temp['ref'] = 'https://esdac.jrc.ec.europa.eu/projects/lucas'
temp['gps_lat'] = nod_lucas['lat']
temp['gps_long'] = nod_lucas['lon']
temp['country'] = nod_lucas['nuts0'] + '-lucas'
df_nod = temp
print(len(temp))

# with depth different
column_names = ['ph_h2o','ph_ca','oc','gps_lat','gps_long','time','hzn_top','hzn_bottom','ref','ph_kcl']
temp = pd.DataFrame(columns=column_names)
temp['oc'] = depth_lucas['oc']
temp['ph_kcl'] = np.nan
temp['ph_h2o'] = depth_lucas['pH_in_H2O']
temp['ph_ca'] = depth_lucas['pH_in_CaCl2']
temp['time'] = depth_lucas['time']
temp['hzn_top'] = depth_lucas['depth'].str.split('-').str[0].astype(float)
temp['hzn_bottom'] = depth_lucas['depth'].str.split('-').str[-1].str[0:-2].astype(float)
temp['ref'] = 'https://esdac.jrc.ec.europa.eu/projects/lucas'
temp['gps_lat'] = depth_lucas['lat']
temp['gps_long'] = depth_lucas['lon']
temp['country'] = depth_lucas['nuts0'] + '-lucas'
df_dep = temp
print(len(temp))

dff = pd.concat([df_nod,df_dep])

l1 = len(dff)
dff = dff.dropna(subset=['gps_lat','gps_long'])
print(f'- {l1-len(dff)}, data with no coor info')

43862
19125
- 110, data with no coor info


In [44]:
# lucas swiss
swiss = pd.read_csv('/home/opengeohub/xuemeng/work_xuemeng/delta_soc/LUCAS_TOPSOIL/LUCAS2015_topsoildata_20200323/LUCAS_2015_Topsoil_data_of_Switzerland-with-coordinates.csv')
swiss = swiss.iloc[:, :-12] # exclude the other method
swiss.columns = swiss.iloc[0] # get the correct column names
swiss = swiss.loc[1::]

column_names = ['ph_h2o','ph_ca','oc','gps_lat','gps_long','time','hzn_top','hzn_bottom','ref','ph_kcl']
temp = pd.DataFrame(columns=column_names)
temp['oc'] = swiss['Organic carbon/\ng kg–1']
temp['ph_kcl'] = np.nan
temp['ph_h2o'] = swiss['pH in H2O\n']
temp['ph_ca'] = swiss['pH in CaCl2']
temp['time'] = 2015
temp['hzn_top'] = 0
temp['hzn_bottom'] = 20
temp['ref'] = 'https://esdac.jrc.ec.europa.eu/content/lucas-2015-topsoil-data-switzerland'
temp['gps_lat'] = swiss['GPS_ LAT']
temp['gps_long'] = swiss['GPS_LONG']
temp['country'] = 'swiss'
dff = pd.concat([dff,temp])

In [45]:
dff.loc[dff['time'].isna()]

,ph_h2o,ph_ca,oc,gps_lat,gps_long,time,hzn_top,hzn_bottom,ref,ph_kcl,country


In [46]:
dff.to_csv('training_points_v2_lucas.full.csv', index = False)